In [6]:
import numpy as np
from moviepy.editor import VideoFileClip, concatenate
from moviepy.editor import *

import matplotlib.pyplot as plt

In [8]:
input_file = 'video.mp4'

In [9]:
clip = VideoFileClip(input_file).subclip(0,100)

In [10]:
fps = 44000
a = 1
b = 10
out_name = 'cut' + '.mp4'

In [11]:
y = clip.audio.subclip(10,20).to_soundarray(fps=10000)[:,0] # only get one of the soundvawes

In [12]:
y = clip.audio.to_soundarray(fps=fps)[:,0] # only get one of the soundvawes
y = np.abs(y)

In [13]:
def get_non_silent_parts(a, b, silence=0.01):
    a_frames = int(a * fps)

    # whether frames from i*a_frames:(i+1)*a_frames are sounded
    # which is equal to i-seconds:i+a-seconds
    sounds = []

    current = 0

    for i in range(0,len(y),a_frames): # len(y)
        interval = y[i : i + a_frames]

        split_intervals = np.array(np.split(interval,b))

        means = split_intervals.mean(axis=1)

        silent = means < silence


        new_silent = np.all(silent)

        if not current:
            new_silent == 0

        sounds.append(new_silent)

        current = new_silent

        """ plt.plot(interval)
        plt.plot([(x+1/2)*a_frames/b for x in range(b)], means) """

    return np.invert(sounds)

In [14]:
def getSecondsTuples():
    sounds_array = get_non_silent_parts(a,b)

    areas = []

    current = 0
    current_start = 0

    for i in range(len(sounds_array)):
        if sounds_array[i] == 1:
            if current != 1:
                current_start = i
                current = 1
        else:
            if current == 1:
                areas.append((current_start*a,(i-1)*a))
            current = 0
    return areas

In [15]:
def makeNewVideo():
    sounds = get_non_silent_parts(a,b)
    timestamps = np.arange(0,clip.duration,a)
    new_videos = np.array([clip.subclip(i,i+a) for i,x in zip(timestamps,sounds) if x])
    return concatenate(new_videos)

In [16]:
makeNewVideo().write_videofile(out_name, threads = 4,logger = None)